<a href="https://colab.research.google.com/github/JacopoMangiavacchi/Swift-TensorFlow-Sample-Notebooks/blob/master/FizzBuzz_Swift_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import Foundation
import TensorFlow

In [0]:
func bitEncode(_ n: UInt, bits: Int = 10) -> [Int32] {
    var array = [Int32](repeating: 0, count: bits)
    for i in 0..<bits {
        array[bits - 1 - i] = Int32(n >> i & 1)
    }
    return array
}

func labelEncode(_ n: UInt) -> [Int32] {
    var array: [Int32] = [0, 0, 0, 0]
    guard n > 0 else { return array }

    if n % 15 == 0 {
        array[0] = 1
    }
    else if n % 3 == 0 {
        array[1] = 1
    }
    else if n % 5 == 0 {
        array[2] = 1
    }
    else {
        array[3] = 1
    }

    return array
}

In [0]:
let start = 101
let end = 1024
var X = [[Int32]]()
var Y = [[Int32]]()

for i in start..<end {
    X.append(bitEncode(UInt(i)))
    Y.append(labelEncode(UInt(i)))
}

In [0]:
struct FizzBuzz: Layer {
    var l1: Dense<Float>
    var l2: Dense<Float>

    init(variables: Int = 1) {
        l1 = Dense<Float>(inputSize: variables, outputSize: 50, activation: relu)
        l2 = Dense<Float>(inputSize: 50, outputSize: 4, activation: softmax)
    }
    
    @differentiable(wrt: (self, input))
    func applied(to input: Tensor<Float>) -> Tensor<Float> {
        let output = l1.applied(to: input)
        return l2.applied(to: output)

    }
}

In [24]:
let x = Tensor<Int32>(shape: [Int32(end - start), 10], scalars: Array(X.joined()))
let y = Tensor<Int32>(shape: [Int32(end - start), 4], scalars: Array(Y.joined()))

print(x.shape)
print(y.shape)

TensorShape(dimensions: [923, 10])
TensorShape(dimensions: [923, 4])


In [0]:
let optimizer = SGD<LinearRegression, Float>(learningRate: 0.03)
var model = LinearRegression(variables: 3)


for epoch in 1...3000 {
    let (cost, 𝛁model) = model.valueWithGradient { m -> Tensor<Float> in
        let ŷ = m.applied(to: x)
        return meanSquaredError(predicted: ŷ, expected: y)
    }
    optimizer.update(&model.allDifferentiableVariables, along: 𝛁model)
  
    if epoch % 1000 == 0 {
        print("Epoch: \(epoch) Cost: \(cost)")
    }
}

Epoch: 1000 Cost: 0.004677016
Epoch: 2000 Cost: 0.003017218
Epoch: 3000 Cost: 0.002556147


In [0]:
let height = heightNormalizer.normalize(Float(76.8))
let heightTensor = TensorElementLiteral<Float>(floatLiteral: height)

let normPredictedWeight = model.applied(to: [[heightTensor, 0, 1]])  //Height, Female, Male
let predictedWeight = weightNormalizer.denormalize(Float(normPredictedWeight[0][0])!)

print(predictedWeight)

225.99329
